In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
rand_y = np.abs(np.random.normal(0,10, size = (20000,16)))
rand_y = rand_y.round(0)
rand_y[rand_y > 28] = 28
rand_y = pd.DataFrame(rand_y, columns = ('Ax', 'Bx', 'By', 'Cy', 'Cx', 'Dx', 'Dy', 'Ay', 'AQ', 'AL', 'BQ', 'BL', 'CQ', 'CL', 'DQ', 'DL'))
rand_y = rand_y.astype('str')
rand_y['Ax'] = 'numb_dele(1) = ' + rand_y['Ax']
eof = pd.DataFrame(columns = rand_y.columns)
rand_y.loc[1::2] = '/eof'
rand_y.to_csv('rand_y.txt',index = False, index_label = False)

In [ ]:
y = pd.read_table(os.path.join('data', 'data_output.txt'),sep='\s+', engine='python', dtype ='Int64')
X = pd.read_table(os.path.join('data', 'data_input.txt'),sep='\s+', engine='python', dtype ="Float64")

In [ ]:
ind1_list = []
ind2_list = []
for i in range(X.shape[0]):
    ind1_list.append(i//4) 
    ind2_list.append(i%4+1)
mInd = pd.MultiIndex.from_arrays([ind1_list, ind2_list], names=('number', 'mode'))
X.index = mInd

In [ ]:
X = np.abs(X.copy())
n1 = X.columns[1:9:2]
n2 = X.columns[2:9:2]
formula = n1 + n2 + ' = (' + n1 + ' + ' + n2 + ')/2'

In [ ]:
def norm_frez(x):
    #x['frez'].iloc[3] = x['frez'].iloc[3] - 14.7  
    f_deltamin = x['frez'] - np.min(x['frez'])
    f_norm = f_deltamin/np.max(f_deltamin)
    x['frez'] = f_norm.round(3)
    return x  

In [ ]:
X = X.groupby(level = 'number').apply(norm_frez)

In [ ]:
for i in range(4): X = X.eval(formula[i]).drop(columns = [n1[i],n2[i]]) 
X_frez = X.pop('frez')
maxval = np.max(X, axis = 1)
X = X.divide(maxval, axis = 0, level=1)
X = X.round(3)
X.insert(0,'frez', X_frez)

In [ ]:
ind = []
for i in range(X.index[-1][0]):
    Xstart = X.loc[i].drop(columns = 'frez')
    for j in (range((i + 1),X.index[-1][0])):
        Xnext = X.loc[j].drop(columns = 'frez')
        if (np.mean(np.abs(Xnext - Xstart), axis = 1) < 0.05).all():
            ind.append([i,j])      

In [ ]:
ident = y.astype(int)

In [ ]:
ident['AxBx']= ident['Ax'] + ident['Bx']
ident['ByCy']= ident['By'] + ident['Cy']
ident['CxDx']= ident['Cx'] + ident['Dx']
ident['DyAy']= ident['Dy'] + ident['Ay']
ident = ident.drop(columns = ['Ax','Bx','By','Cy','Cx','Dx','Dy','Ay'])

In [ ]:
def in_all(x, **kwargs):
    for key in kwargs.keys():
        values = kwargs[key]
        Lva = ['AxBx','ByCy','CxDx','DyAy']
        if (x[key] != 0) and (x[values[0]] != 0 or x[values[1]] != 0):
            if (x[values[0]] == 0):
                val = values[1]
            elif (x[values[1]] == 0):
                val = values[0]
            else:
                print('Error')
                break  
            minval = np.floor(np.min([x[val],x[key]/2]))
            x[key] = x[key] - 2*minval
            if 'x' in val:
                x['X'] += 2*minval
                direct = 'y'       
            elif 'y' in val:
                x['Y'] += 2*minval
                direct = 'x'
            else:
                print('Error')
            x[val] = 0
            search = key + direct
            for i in Lva:
                if search in i:
                     x[i] = minval
    return x    

In [ ]:
def end_step(x, **kwargs):
    for key in kwargs.keys():
        values = kwargs[key]
        x[values] = x[values] + x[key] // 2 
        x[key] = x[key] % 2 
    return x

In [ ]:
def in_cont(x, **kwargs):
    for key in kwargs.keys():
        values = kwargs[key]
        minval = np.min(x[values])
        x[values] = x[values] - minval
        if key in x:
            x[key] = x[key] + len(values)*minval
        else:
            x[key] = len(values)*minval
    return x

In [ ]:
def end_end_step(x):
    alist = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL']
    ac = []
    bd = []
    for a in alist:
        if x[a] != 0:
            if ('A' in a) or ('C' in a):
                ac.append(a)
            elif('B' in a) or ('D' in a):
                bd.append(a)
    if (len(ac) > 0):
        con = x['CQ'] + x['CL']
        x['C'] = x['C'] + con*2
        if ((x['AQ'])!= 0) or ((x['CL'])!= 0):
            x['CQ'] = x['CL'] = 0
            sa = 'Q' 
            x['L'] =  x['L'] + con*2
            x['AQ'] = x['AQ'] + con
        elif ((x['AL'])!= 0) or ((x['CQ'])!= 0):
            x['CQ'] = x['CL'] = 0
            sa = 'L'
            x['Q'] = x['Q'] + con*2
            x['AL'] = x['AL'] + con
    elif (len(ac) == 0):
        sa = ''
    if (len(bd) > 0):
        if ('D' + sa) in bd:
            con = x['DQ'] + x['DL']
            x['D'] = x['D'] + con*2
            if ((x['BQ'])!= 0) or ((x['DL'])!= 0):
                x['DQ'] = x['DL'] = 0
                x['Q'] =  x['Q'] + con*2
                x['BQ'] = x['BQ'] + con 
            elif ((x['BL'])!= 0) or ((x['DQ'])!= 0):
                x['DQ'] = x['DL'] = 0
                x['L'] = x['L'] + con*2
                x['BL'] = x['BL'] + con 
        elif ('B' + sa) in bd:
            con = x['BQ'] + x['BL']
            x['B'] = x['B'] + con*2
            if ((x['DQ'])!= 0) or ((x['BL'])!= 0):
                x['BQ'] = x['BL'] = 0
                x['Q'] =  x['Q'] + con*2
                x['DQ'] = x['DQ'] + con 
            elif ((x['DL'])!= 0) or ((x['BQ'])!= 0):
                x['BQ'] = x['BL'] = 0
                x['L'] =  x['L'] + con*2
                x['DL'] = x['DL'] + con          
    return x    

In [ ]:
def in_XY(x, **kwargs):
    for key in kwargs.keys():
        if x[key] != 0:
            values = kwargs[key]
            var = x[key]/4
            x[values[0]] = x[values[0]] + 2*var
            x[values[1]] = x[values[1]] + var
            x[values[2]] = x[values[2]] + var
            x[key] = 0
    return x    

In [ ]:
pairs = {'AxBx':['Ax','Bx'], 'ByCy':['By','Cy'], 'CxDx':['Cx','Dx'], 'DyAy':['Dy','Ay']}
direction = {'X': ['AxBx','CxDx'],'Y': ['ByCy','DyAy'],'Q':['AQ','BL','CQ','DL'],'L':['AL','BQ','CL','DQ']}
massXY = {'A': ['AxBx','DyAy'],'B': ['AxBx','ByCy'],'C': ['CxDx','ByCy'], 'D': ['CxDx','DyAy']}
massQL = {'A': ['AQ','AL'],'B': ['BQ','BL'], 'C': ['CQ','CL'], 'D': ['DQ','DL']}
delXY = {'del': ['X','Y']}
delQL = {'del': ['Q','L']}
delAC = {'del': ['A','C']}
delBD = {'del': ['B','D']}
contM = {'AB': ['A','B'],'BC': ['B','C'],'CD': ['C','D'],'DA': ['D','A'] }
contM1 = {'AB': ['AxBx','DyAy','ByCy'],'BC': ['ByCy','AxBx','CxDx'],
         'CD': ['CxDx','ByCy','DyAy'],'DA': ['DyAy','CxDx','AxBx'] }
allM = {'A':['ByCy','CxDx'],'B':['CxDx','DyAy'],'C':['AxBx','DyAy'],'D':['AxBx','ByCy']}
end1 = {'Q':['AQ','CQ'],'L':['BL','DL']}
end2 = {'Q':['BQ','DQ'],'L':['AL','CL']}
end3 = {'A':['AQ','AL'],'B':['BQ','BL'],'C':['CQ','CL'],'D':['DQ','DL']}

In [ ]:
ident = ident.apply(in_cont, **direction, axis = 1)
ident = ident.apply(in_cont, **massXY, axis = 1)
ident = ident.apply(in_cont, **massQL, axis = 1)
ident = ident.apply(in_cont, **delXY, axis = 1)
ident = ident.apply(in_cont, **delQL, axis = 1)
ident = ident.apply(in_cont, **delAC, axis = 1)
ident = ident.apply(in_cont, **delBD, axis = 1)
ident = ident.drop(columns = ['del'])
ident = ident.apply(in_cont, **contM, axis = 1)
ident = ident.apply(in_XY, **contM1, axis = 1)
ident = ident.drop(columns = ['AB','BC','CD','DA'])
ident = ident.apply(in_cont, **direction, axis = 1)
ident = ident.apply(in_cont, **delXY, axis = 1)
ident = ident.drop(columns = ['del'])
ident = ident.apply(in_all, **allM, axis = 1)
ident = ident.apply(in_cont, **delXY, axis = 1)
ident = ident.drop(columns = ['del'])
ident = ident.apply(end_step, **end3, axis = 1)
ident = ident.apply(in_cont, **end1, axis = 1)
ident = ident.apply(in_cont, **end2, axis = 1)
ident = ident.apply(in_cont, **massQL, axis = 1)
ident = ident.apply(end_end_step, axis = 1)

In [ ]:
ident = ident.apply(in_all, **allM, axis = 1)

In [ ]:
np.sum(ident,axis = 1)

In [ ]:
[[74, 309], [141, 219], [170, 347], [293, 450]]